In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
import random
import keras_tuner

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
seed = 6
batch_size = 64
input_shape = (96, 96, 3)
epochs = 200
fine_tuning_locked = 12

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset_dir_train = '../data5/training'
dataset_dir_val = '../data5/validation'

labels = ['Species1',       # 0
          'Species2',       # 1
          'Species3',       # 2
          'Species4',       # 3
          'Species5',       # 4
          'Species6',       # 5
          'Species7',       # 6
          'Species8',       # 7
          ]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=50,
                                        height_shift_range=5,
                                        width_shift_range=5,
                                        zoom_range=0.5,
                                        shear_range = 0.1,
                                        brightness_range=[0.4, 1.5],
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect',
                                        rescale=1/255.)

train_val_data_gen = ImageDataGenerator(rescale=1/255.)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method to create iterator over the indices
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir_train,
                                                        target_size=(96, 96),
                                                        color_mode='rgb',
                                                        classes=None,
                                                        class_mode='categorical',
                                                        batch_size=batch_size,
                                                        shuffle=True,#at the end of the epoch it will shuffle the data
                                                        seed=seed,
                                                        )

val_gen = train_val_data_gen.flow_from_directory(directory=dataset_dir_val,
                                                  target_size=(96, 96),
                                                  color_mode='rgb',
                                                  classes=None,
                                                  class_mode='categorical',
                                                  batch_size=batch_size,
                                                  shuffle=True, 
                                                  seed=seed,
                                                  )


In [ ]:
# Download and plot the Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    classes=8, # number of classes to classify images into
    classifier_activation="softmax",
)

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten(name='Flatten')(x)
x = tfkl.Dropout(0.1, seed=seed)(x)
x = tfkl.Dense(units=256, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer=l2(0.0001),activation='relu')(x)
x = tfkl.Dropout(0.1, seed=seed)(x)
output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed),kernel_regularizer=l2(0.0001), name='output_layer')(x)

model = tfk.Model(inputs=base_model.inputs, outputs=output_layer, name='model')
    # Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
model.summary()

In [ ]:
class_weights = {0: 2.389358108108108, 
                 1: 0.8320588235294117, 
                 2: 0.8583131067961165, 
                 3: 0.8667279411764706, 
                 4: 0.8340212264150944, 
                 5: 1.9978813559322033, 
                 6: 0.8243006993006993, 
                 7: 0.8709975369458128}

In [ ]:
callbacks = []

# Tensorboard
tensorboard = TensorBoard(log_dir="tb_logs/")
callbacks.append(tensorboard)
# Early Stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
callbacks.append(es_callback)

# Train the model
model.fit(
    x=aug_train_gen,
    batch_size=batch_size,
    epochs=epochs,
    class_weight=class_weights,
    validation_data=val_gen,
    callbacks= callbacks)

model.save("testing_x/CNN_xception_3.0_fine_tuning_12")

In [ ]:
#fine tuning
del model
model = tfk.models.load_model('testing_x/CNN_xception_3.0_fine_tuning_12')
model.summary()

In [ ]:
for layer in base_model.layers[:fine_tuning_locked]:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Fine-tune the model
model.fit(
    x = aug_train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=val_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
)

In [ ]:
model.save('testing_x/FineTuningModel_12')